## Generating C code for the right-hand-side of the scalar wave equation, in Cartesian coordinates

### Problem Statement

We wish to numerically solve the scalar wave equation in Cartesian coordinates:
$$\partial_t^2 u = c^2 \nabla^2 u \text{,}$$
where $u$ is a function of time and space: $u = u(t,x,y,...)$ (spatial dimension as-yet unspecified) and $c$ is the wave speed,

subject to some initial condition
$$u(0,x,y,...) = f(x,y,...)$$

and suitable, approximate spatial boundary conditions.

Define 
$$v(t,x,y,...) = \partial_t u(t,x,y,...)$$

In this way, the second-order PDE is reduced to a set of two coupled first-order PDEs

\begin{align}
\partial_t u &= v \\
\partial_t v &= c^2 \nabla^2 u.
\end{align}

**Our goal will be to generate C code capable of computing the right-hand sides of the above equation.**

### Numerical Solution to the Wave Equation 

We will find that breaking the second-order time derivative into two first-order, coupled equations in this way is quite convenient numerically, as it enables us to step forward in time using the Method of Lines. This method enables us to handle timestepping the same way we would handle an ODE integration, allowing us to use [Runge-Kutta methods](https://en.wikipedia.org/wiki/Runge%E2%80%93Kutta_methods) for timestepping. 

As for the $\nabla^2 u$ term, spatial derivatives are handled in NRPy+ via [finite differencing](https://en.wikipedia.org/wiki/Finite_difference).

We will sample the solution $\{u,v\}$ at discrete, uniformly-sampled points in space and time For simplicity, let's assume that we consider the wave equation in one spatial dimension. Then the solution at any sampled point in space and time is given by
$$u^n_i = u(t_n,x_i) = u(t_0 + n \Delta t, x_0 + i \Delta x),$$
where $\Delta t$ and $\Delta x$ represent the temporal and spatial resolution, respectively. $v^n_i$ is sampled at the same points in space and time.

#### Spatial Derivatives: the $\nabla^2$ Operator

To minimize complication, we will restrict ourselves to solving the wave equation in one spatial dimension, so
$$\nabla^2 u = \partial_x^2 u.$$
Extension of this operator to higher spatial dimensions is straightforward, particularly when using NRPy+.

As was discussed in [the finite difference section of the tutorial](Tutorial-Finite_Difference_Derivatives.ipynb), NRPy+ approximates derivatives using [finite difference methods](),  the second-order derivative $\partial_x^2$ accurate to fourth-order in uniform grid spacing $\Delta x$ (from fitting the unique 4th-degree polynomial to 5 sample points of $u$) is given by
\begin{equation}
\left[\partial_x^2 u(t,x)\right]_j = \frac{1}{(\Delta x)^2}
\left(
-\frac{1}{12} \left(u_{j+2} + u_{j-2}\right) 
+ \frac{4}{3}  \left(u_{j+1} + u_{j-1}\right)
- \frac{5}{2} u_j \right)
+ \mathcal{O}\left((\Delta x)^4\right).
\end{equation}

In [1]:
import NRPy_param_funcs as par
import indexedexp as ixp
import grid as gri
import finite_difference as fin
from outputC import *

In [2]:
# The name of this module ("scalarwave") is given by __name__:
thismodule = __name__

# Step 0: Set the spatial dimension parameter, and then read
#         the parameter as DIM.
par.set_parval_from_str("grid::DIM",1)
DIM = par.parval_from_str("grid::DIM")

# Step 1: Set the finite differencing order to 4.
par.set_parval_from_str("finite_difference::FD_CENTDERIVS_ORDER",4)

# Step 2: Register gridfunctions that are needed as input.
uu, vv = gri.register_gridfunctions("EVOL",["uu","vv"])

# Step 3: Declare the rank-2 indexed expression \partial_{ij} u,
#         which is symmetric about interchange of indices i and j
#         Derivative variables like these must have an underscore
#         in them, so the finite difference module can parse the
#         variable name properly.
uu_dDD = ixp.declarerank2("uu_dDD","sym12")

# Step 4: Define the C parameter wavespeed. The `wavespeed`
#         variable is a proper SymPy variable, so it can be
#         used in below expressions. In the C code, it acts
#         just like a usual parameter, whose value is 
#         specified in the parameter file.
wavespeed = par.Cparameters("REAL",thismodule,"wavespeed")

# Step 5: Define right-hand sides for the evolution.
uu_rhs = vv
vv_rhs = 0
for i in range(DIM):
    vv_rhs += wavespeed*wavespeed*uu_dDD[i][i]

vv_rhs = sp.simplify(vv_rhs)
    
# Step 6: Generate C code for scalarwave evolution equations,
#         print output to the screen (standard out, or stdout).
fin.FD_outputC("stdout",
               [lhrh(lhs=gri.gfaccess("rhs_gfs","uu"),rhs=uu_rhs),
                lhrh(lhs=gri.gfaccess("rhs_gfs","vv"),rhs=vv_rhs)])

/*
 *  Original SymPy expressions:
 *  "[const double uu_dDD00 = invdx0**2*(-5*uu/2 + 4*uu_i0m1/3 - uu_i0m2/12 + 4*uu_i0p1/3 - uu_i0p2/12),
 *    rhs_gfs[IDX2(UUGF, i0)] = vv,
 *    rhs_gfs[IDX2(VVGF, i0)] = uu_dDD00*wavespeed**2]"
 */
{
   const double uu_i0m2 = in_gfs[IDX2(UUGF, i0-2)];
   const double uu_i0m1 = in_gfs[IDX2(UUGF, i0-1)];
   const double uu = in_gfs[IDX2(UUGF, i0)];
   const double uu_i0p1 = in_gfs[IDX2(UUGF, i0+1)];
   const double uu_i0p2 = in_gfs[IDX2(UUGF, i0+2)];
   const double vv = in_gfs[IDX2(VVGF, i0)];
   const double uu_dDD00 = pow(invdx0, 2)*(-(5.0 / 2.0)*uu + ((4.0 / 3.0))*uu_i0m1 - (1.0 / 12.0)*uu_i0m2 + ((4.0 / 3.0))*uu_i0p1 - (1.0 / 12.0)*uu_i0p2);
   rhs_gfs[IDX2(UUGF, i0)] = vv;
   rhs_gfs[IDX2(VVGF, i0)] = uu_dDD00*pow(wavespeed, 2);
}


**Success!** Notice that indeed NRPy+ was able to compute the spatial derivative operator,
\begin{equation}
\left[\partial_x^2 u(t,x)\right]_j \approx \frac{1}{(\Delta x)^2}
\left(
-\frac{1}{12} \left(u_{j+2} + u_{j-2}\right) 
+ \frac{4}{3}  \left(u_{j+1} + u_{j-1}\right)
- \frac{5}{2} u_j \right)
\end{equation}
correctly (easier to read in the "Original SymPy expressions" comment block at the top of the C output.

Now let's repeat the process, except at *10th* finite difference order, for the *4+1D* scalar wave equation, with SIMD enabled:

In [ ]:
# Step -1: Enable SIMD
par.set_parval_from_str("outputC::SIMD_enable",True)

# Step 0: Set the spatial dimension parameter, and then read
#         the parameter as DIM.
par.set_parval_from_str("grid::DIM",4)
DIM = par.parval_from_str("grid::DIM")

# Step 1: Set the finite differencing order to 10.
par.set_parval_from_str("finite_difference::FD_CENTDERIVS_ORDER",10)

# Step 2: Register gridfunctions that are needed as input.
# Already registered in the previous step!
#uu, vv = gri.register_gridfunctions("EVOL",["uu","vv"])

# Step 3: Declare the rank-2 indexed expression \partial_{ij} u,
#         which is symmetric about interchange of indices i and j
#         Derivative variables like these must have an underscore
#         in them, so the finite difference module can parse the
#         variable name properly.
uu_dDD = ixp.declarerank2("uu_dDD","sym12")

# Step 4: Define the C parameter wavespeed. The `wavespeed`
#         variable is a proper SymPy variable, so it can be
#         used in below expressions. In the C code, it acts
#         just like a usual parameter, whose value is 
#         specified in the parameter file.
# Already initialized:
#wavespeed = par.Cparameters("REAL",thismodule,"wavespeed")

# Step 5: Define right-hand sides for the evolution.
uu_rhs = vv
vv_rhs = 0
for i in range(DIM):
    vv_rhs += wavespeed*wavespeed*uu_dDD[i][i]

vv_rhs = sp.simplify(vv_rhs)
    
# Step 6: Generate C code for scalarwave evolution equations,
#         print output to the screen (standard out, or stdout).
fin.FD_outputC("stdout",
               [lhrh(lhs=gri.gfaccess("rhs_gfs","uu"),rhs=uu_rhs),
                lhrh(lhs=gri.gfaccess("rhs_gfs","vv"),rhs=vv_rhs)])